## A simple model for demand and supply of publicly-provided services in a city

### Demand modelling

In [ ]:
from enum import Enum
import os.path

import numpy as np
import pandas as pd
import geopandas as gpd
import geopy, geopy.distance
import shapely
from sklearn import gaussian_process

from matplotlib import pyplot as plt 
import seaborn as sns
plt.rcParams['figure.figsize']= (20,14)

In [ ]:
## TODO: find way to put this into some global settings
import os
import sys
nb_dir = os.path.dirname(os.getcwd())
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from references import common_cfg

In [ ]:
from src.models.city_items import AgeGroup, ServiceArea, ServiceType, SummaryNorm # enum classes for the model

In [ ]:
from src.models.process_tools import MappedPositionsFrame

In [ ]:
AgeGroup.classify_array(range(18))

In [ ]:
gaussKern = gaussian_process.kernels.RBF
get_random_pos = lambda n: list(map(geopy.Point, list(zip(np.round(np.random.uniform(45.40, 45.50, n), 5), 
                                np.round(np.random.uniform(9.1, 9.3, n), 5)))))
make_shapely_point = lambda geoPoint: shapely.geometry.Point((geoPoint.longitude, geoPoint.latitude))

In [ ]:
### Demand modelling
class DemandFrame(pd.DataFrame):
    '''A class to store demand units in row and 
    make them available for aggregation'''
    
    def __init__(self, dfIn, bDuplicatesCheck=True):
        assert isinstance(dfIn, pd.DataFrame), 'Input DataFrame expected'
        self.__dict__.update(dfIn.__dict__)
        
        # prepare the AgeGroups cardinalities
        groupsCol = 'ageGroup'
        peopleBySampleAge = common_cfg.fill_sample_ages_in_cpa_columns(self)
        dataByGroup = peopleBySampleAge.rename(AgeGroup.find_AgeGroup, axis='columns').T
        dataByGroup.index.name = groupsCol # index is now given by AgeGroup items
        dataByGroup = dataByGroup.reset_index() # extract to convert to categorical and groupby
        dataByGroup[groupsCol] = dataByGroup[groupsCol].astype('category')
        agesBySection = dataByGroup.groupby(groupsCol).sum()
        self['Ages'] = pd.Series(agesBySection.to_dict()) # assign dict to each section
        self['TotalPeople'] = agesBySection.T.sum(axis=1)
        
        self['Positions'] = self['geometry'].apply(
            lambda pos: geopy.Point(pos.centroid.y, pos.centroid.x))
        
        if bDuplicatesCheck:
            # check no location is repeated - takes a while
            assert not any(self['Positions'].duplicated()), 'Repeated position found'
            
    @property
    def mappedPositions(self):
        return MappedPositionsFrame(positions=self['Positions'].tolist(),
            idQuartiere=self[common_cfg.IdQuartiereColName].tolist())
    
    @property
    def totalPeople(self):
        return sum(self['TotalPeople'])
    
    @staticmethod
    def create_from_istat_cpa(cityName):
        '''Constructor caller for DemandFrame'''
        assert cityName in common_cfg.cityList, \
            'Unrecognised city name "%s"' % cityName
        frame = DemandFrame(common_cfg.get_istat_cpa_data(cityName),
                          bDuplicatesCheck=False)
        return frame

In [ ]:
bb = DemandFrame.create_from_istat_cpa('Milano')

In [ ]:
bb.mappedPositions

In [ ]:
def evaluate_demand(householdList, outputServices= [t for t in ServiceType]):
    """ """
    # initialise output
    outDemand = dict()
    # consolidate positions. If two households share the same position, sum components.
    householdData = pd.concat([h.export for h in householdList])
    householdData['position'] = householdData.index 
    consolidated = householdData.groupby('position').sum()
    
    for thisServType in outputServices:
        outDemand[thisServType] = consolidated*thisServType.demandFactors
        
    return outDemand